In [2]:
import findspark
findspark.init('/home/duynguyen/spark-master')

In [108]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
# $example off$
sc = SparkContext(appName="PythonSVDExample")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PythonSVDExample, master=local[*]) created by __init__ at <ipython-input-4-f8992841a1cc>:2 

In [106]:
if __name__ == "__main__":
    sc = SparkContext(appName="PythonSVDExample")

    # $example on$
    rows = sc.parallelize([
#         Vectors.sparse(5, {1: 1.0, 3: 7.0}),
        Vectors.dense(1.0, 5.0, 0.0, 2.0, 3.0),
        Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
        Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
    ])

    mat = RowMatrix(rows)

    # Compute the top 5 singular values and corresponding singular vectors.
    svd = mat.computeSVD(5, computeU=True)
    U = svd.U       # The U factor is a RowMatrix.
    s = svd.s       # The singular values are stored in a local dense vector.
    V = svd.V       # The V factor is a local dense matrix.
    # $example off$
    collected = U.rows.collect()
    print("U factor is:")
    for vector in collected:
        print(vector)
    print("Singular values are: %s" % s)
    print("V factor is:\n%s" % V)
# sc.stop()

U factor is:
[-0.335408389214,-0.940513917832,-0.0541736357324,7.45058059692e-09,-1.49011611938e-08]
[-0.543122633482,0.240035423048,-0.804612205154,0.0,-7.45058059692e-09]
[-0.769752568976,0.240450755968,0.591324290478,0.0,-2.98023223877e-08]
Singular values are: [12.8217050658,4.81250330918,2.53844265354,9.17848613194e-08,5.12422751202e-08]
V factor is:
DenseMatrix([[-0.35101914,  0.10417862,  0.27650777,  0.87978099,  0.12434323],
             [-0.13079711, -0.97715665, -0.10670644,  0.10722581, -0.07192613],
             [-0.12707888,  0.14963237, -0.95091241,  0.23598501, -0.03921457],
             [-0.58196805,  0.10842972,  0.08712021, -0.16150041, -0.78478269],
             [-0.71052222,  0.01283961, -0.01824422, -0.36430307,  0.6016176 ]])


In [5]:
B = mat.rows.collect()

In [6]:
import numpy as np

In [7]:
Norm_A = np.linalg.norm(V,ord='fro')

ValueError: Improper number of dimensions to norm.

In [109]:
import numpy as np
from pyspark.mllib.linalg.distributed import RowMatrix
C = np.arange(1024 ** 2, dtype=np.float64).reshape(1024, 1024)
D = np.arange(1024 ** 2, dtype=np.float64).reshape(1024, 1024)

from pyspark.mllib.linalg.distributed import *

def as_block_matrix(rdd, rowsPerBlock=1024, colsPerBlock=1024):
    return IndexedRowMatrix(
        rdd.zipWithIndex().map(lambda xi: IndexedRow(xi[1], xi[0]))
    ).toBlockMatrix(rowsPerBlock, colsPerBlock)

matrixC = as_block_matrix(sc.parallelize(C))
matrixD = as_block_matrix(sc.parallelize(D))
product = matrixC.multiply(matrixD)

AttributeError: 'PipelinedRDD' object has no attribute 'toDF'

(1024, 1024)

In [29]:
np.dot(V,V)

TypeError: unsupported operand type(s) for *: 'DenseMatrix' and 'DenseMatrix'

In [30]:
temp_s = np.diag(s)

In [31]:
temp_V = V.toArray()

In [104]:
results = np.dot(temp_s,temp_V)

In [35]:
from pyspark.mllib.linalg import Matrices

In [41]:
temp = sc.parallelize(results)

In [42]:
B = RowMatrix(temp)

In [43]:
for vector in B.rows.collect():
    print(vector)

[-4.50066393208,1.33574753618,3.54530106761,11.2802923816,1.59429219727]
[-0.629461516521,-4.70256958916,-0.513525095532,0.516024562279,-0.346144747922]
[-0.322582445593,0.379833186855,-2.41383661546,0.599034405298,-0.0995439420233]
[-5.34158569864e-08,9.95220692798e-09,7.99631616664e-09,-1.48232923944e-08,-7.20311703185e-08]
[-3.64087749578e-08,6.57930699292e-10,-9.34875207151e-10,-1.86677183883e-08,3.08282547557e-08]


In [45]:
results

array([[ -4.50066393e+00,   1.33574754e+00,   3.54530107e+00,
          1.12802924e+01,   1.59429220e+00],
       [ -6.29461517e-01,  -4.70256959e+00,  -5.13525096e-01,
          5.16024562e-01,  -3.46144748e-01],
       [ -3.22582446e-01,   3.79833187e-01,  -2.41383662e+00,
          5.99034405e-01,  -9.95439420e-02],
       [ -5.34158570e-08,   9.95220693e-09,   7.99631617e-09,
         -1.48232924e-08,  -7.20311703e-08],
       [ -3.64087750e-08,   6.57930699e-10,  -9.34875207e-10,
         -1.86677184e-08,   3.08282548e-08]])

In [46]:
B = RowMatrix(sc.parallelize(np.dot(np.diag(s),V.toArray())))

In [47]:
for vector in B.rows.collect():
    print(vector)

[-4.50066393208,1.33574753618,3.54530106761,11.2802923816,1.59429219727]
[-0.629461516521,-4.70256958916,-0.513525095532,0.516024562279,-0.346144747922]
[-0.322582445593,0.379833186855,-2.41383661546,0.599034405298,-0.0995439420233]
[-5.34158569864e-08,9.95220692798e-09,7.99631616664e-09,-1.48232923944e-08,-7.20311703185e-08]
[-3.64087749578e-08,6.57930699292e-10,-9.34875207151e-10,-1.86677183883e-08,3.08282547557e-08]


In [48]:
U.multiply()

TypeError: 'RowMatrix' object does not support indexing

In [56]:
vector1 = Vectors.dense(1.0, 5.0, 0.0, 2.0, 3.0)

In [64]:
vector1.numNonzeros()

4

In [58]:
from numpy import zeros, max, sqrt, isnan, isinf, dot, diag, count_nonzero, where
count_nonzero(vector1)

4

In [60]:
vector2 = Vectors.sparse(5, {1: 1.0, 3: 7.0})

In [63]:
vector2.numNonzeros()

2

In [74]:
temp = mat.rows.collect()

In [90]:
mat.rows.union(mat.rows).collect()

[DenseVector([1.0, 5.0, 0.0, 2.0, 3.0]),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0]),
 DenseVector([1.0, 5.0, 0.0, 2.0, 3.0]),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [83]:
temp

[DenseVector([1.0, 5.0, 0.0, 2.0, 3.0]),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [93]:
sketchMatrix = zeros((5,5))
sketchMatrix

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [98]:
sM = RowMatrix(sc.parallelize(sketchMatrix))

In [99]:
sM.rows.collect()

[DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 0.0, 0.0, 0.0, 0.0])]

In [100]:
del(sM)

In [102]:
svd

In [ ]:
V.